In [1]:
%pip install azure-eventhub

In [13]:
from delta.tables import *
import json
import time

# the following data was sent by the shipping company such as DHL, Purolator

shipping_list = '[                                                                   \
    {"order_number":"2501","order_date":"04/05/2020","shipping_date":"04/10/2020"},  \
    {"order_number":"2502","order_date":"03/29/2017","shipping_date":"04/05/2017"},  \
    {"order_number":"2503","order_date":"07/27/2019","shipping_date":"08/22/2019"},  \
    {"order_number":"2505","order_date":"06/30/2020","shipping_date":"06/30/2020"},  \
    {"order_number":"2506","order_date":"10/28/2018","shipping_date":"10/30/2018"},  \
    {"order_number":"2509","order_date":"09/05/2019","shipping_date":"09/09/2019"},  \
    {"order_number":"2510","order_date":"01/07/2018","shipping_date":"01/22/2018"},  \
    {"order_number":"2511","order_date":"01/18/2020","shipping_date":"01/18/2020"},  \
    {"order_number":"2512","order_date":"09/04/2020","shipping_date":"09/27/2020"},  \
    {"order_number":"2513","order_date":"03/19/2021","shipping_date":""}             \
                ]'

#shipping_list = '[                                                                   \
#    {"order_number":"2516","order_date":"04/05/2020","shipping_date":"04/10/2020"},  \
#    {"order_number":"2517","order_date":"09/04/2020","shipping_date":"09/27/2020"},  \
#    {"order_number":"2518","order_date":"03/19/2021","shipping_date":""}             \
#                ]'

df_shipping = spark.read.json(sc.parallelize([shipping_list]))
df_shipping.createOrReplaceTempView('shipping')
display(df_shipping)
df_shipping.printSchema()


In [9]:
# convert the order_date and shipping_date to date types
df_shipping=spark.sql("""
  SELECT order_number, 
  TO_DATE(CAST(UNIX_TIMESTAMP(order_date, 'MM/dd/yyyy') AS TIMESTAMP)) as order_date,
  TO_DATE(CAST(UNIX_TIMESTAMP(shipping_date, 'MM/dd/yyyy') AS TIMESTAMP)) as shipping_date
  FROM shipping"""
)

df_shipping.createOrReplaceTempView('shipping')
display(df_shipping)
df_shipping.printSchema()

In [10]:
# find the elapsed days for shipping the products
df_late_shipping=spark.sql("""
  SELECT order_number, order_date, shipping_date, datediff(shipping_date, order_date) as shipping_days
  FROM shipping """
)

display(df_late_shipping)

In [12]:

try:
    for shipping in df_late_shipping.toJSON().collect():
        event_data = EventData(shipping)
        eh_producer.send_event(event_data)
        time.sleep(1)
        print(event_data)
    eh_producer.flush()
    print("Message produced!")
except Exception as e:
    print(f"Error producing message: {e}")
finally:
    eh_producer.close()